<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/PDSH-cover-small.png?raw=1">

*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) | [Contents](Index.ipynb) | [Hierarchical Indexing](03.05-Hierarchical-Indexing.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.04-Missing-Values.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


# 處理遺漏值

## 對於遺漏值表示的慣例

1. 用-9999或很少見的數字來表示遺漏值
2. 用特殊的浮點數NaN(Not a Number)或None物件表示





None:是一種object。（None在數值運算中會出現錯誤，ex: sum(),min()，因此出現了NaN這樣的數值缺失資料型態）

NaN:特殊浮點數數值。

## 在Pandas的遺漏值



### ``None``: Python的缺失資料

因為None是Python物件, 不能夠隨意的用在NumPy/Pandas的陣列中，只能夠以資料型態``'object'``(也就是Python object陣列)放在陣列中:

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

在陣列中使用Python物件，代表著當在含有``None``值的陣列間執行運算像是``sum()``或``min()``，會出現執行錯誤:

In [4]:
vals1.sum()

TypeError: ignored

反映出整數和``None``進行加法運算是沒有被定義的

### ``NaN``: 缺失的數值資料

另外一個遺漏值的表示法: ``NaN`` (*Not a Number*)是一個特殊的浮點數值。所有系統只要是使用標準IEEE浮點數表示法就可以辨別:

In [5]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

只要有經過NaN的運算，計算出來的結果都會是NaN

In [6]:
1 + np.nan

nan

In [7]:
0 *  np.nan

nan

注意:NaN的運算不會導致錯誤

In [8]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy提供特殊的聚合運算可以忽略這些遺漏值

In [9]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

NaN是一個特殊的浮點數，並不是整數、字串、或其他型態的NaN值

### 在Pandas中的NaN和None

``NaN`` 和 ``None``都有各自的用處，可在Pandas中交錯使用:

In [10]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Pandas會在NA值出現時自動轉型。
例如:如果設定一個 ``np.nan``到整數陣列，就會自動把它轉換型別到浮點數以適應這個NA值:

In [11]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [12]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

以下列出Pandas中遇到NA值時，型別轉換的慣例:

|型態類別     |儲存NA時的轉換動作 |NA哨兵值      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |


## 在Null值上操作

利用``None``和``NaN``可交錯使用的特性，可在Pandas的資料結構中有許多方法來偵測、移除及取代空值。

- ``isnull()``: 產生布林遮罩以指示遺漏值
- ``notnull()``: 和``isnull()``相反的操作
- ``dropna()``: 傳回一個過濾過版本的資料
- ``fillna()``: 傳回一個含有被填入或估算進遺漏值的資料複本

### 偵測空值(Null)
Pandas的資料結構有2個偵測空資料的方式: ``isnull()`` and ``notnull()``.都會傳回一個在資料上的布林遮罩。

In [13]:
data = pd.Series([1, np.nan, 'hello', None])

In [14]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [15]:
data[data.notnull()] # 也可以當作Series(本例)或Dataframe的索引

0        1
2    hello
dtype: object

### 拋棄空值

移除NA值``dropna()``

In [17]:
data.dropna() #Series

0        1
2    hello
dtype: object

In [18]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]]) # Dataframe
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


無法從 ``DataFrame``中拋棄單一個值，只能拋棄一整列或一整欄。

``dropna()``提供了許多選項給``DataFrame``使用。

預設情況下，``dropna()``會移除*任何*出現null值的那些列:

In [19]:
df.dropna()

,0,1,2
1,2.0,3.0,5


也可以移除NA值所在的不同資料軸；``axis=1``會移除所有存在空值的欄:

In [20]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


避免一次丟掉太多資料，因此可透過 ``how`` 或 ``thresh``參數，可控制要多少數量的空值才丟棄。

`how`的預設值是any, 也就是任何包含空值的列或欄(根據axis關鍵字的設定)都會被移除。
可以指定``how='all'``, 只有全部都是空值的列或欄才會被移除

In [21]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [22]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


要更細緻的調整可用``thresh``參數來指定非空值的個數至少要多少個，該列或欄才會被保留: 

In [23]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


### 填入空值

填入的值可以是0或來自資料的估算或插補值(Imputation)。

可以使用``isnull()``方法當作是遮罩在做這件事，但因為它是Pandas中非常常用的運算，所以Pandas提供了``fillna()``方法，可以傳回一個取代過空值資料的陣列複本。



In [24]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

用0填入NA值

In [25]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

可以指定一個往前填(forward-fill)讓它使用前一個值填入:

In [26]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

或指定往後填(back-fill)去填入下一個值:

In [27]:
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

在``DataFrame``中，可以指定要沿著哪一個軸來進行填補資料

In [28]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [29]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
